# Initialisation

## Imports

In [97]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM, AutoModelForCausalLM , AutoTokenizer, pipeline, RobertaTokenizer, RobertaForMaskedLM, AlbertTokenizer, AlbertModel, AlbertForMaskedLM
import logging
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
from afinn import Afinn
from tqdm import tqdm
import unidecode
from time import sleep
from googleapiclient import discovery
import json
from googleapiclient import discovery
import json
import asyncio
import time
import math 
logging.basicConfig(level=logging.INFO)# OPTIONAL

## MAC Settings

In [98]:
print(f"PyTorch version: {torch.__version__}")
# Set the device      
device = "mps" if torch.backends.mps.is_available() else torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")

PyTorch version: 1.13.1
Using device: mps


## Definitions

In [99]:
#TEMPLATES
#TEMPLATES
EVALUATION_PATH = '../data/evaluation/'
RESULTS_PATH = '../data/results/'
ALBERT_BASE_TEMPLATE_1 = 'albert-base-v2_template_1.csv'
ALBERT_BASE_TEMPLATE_5 = 'albert-base-v2_template_5.csv'
ALBERT_LARGE_TEMPLATE_1 = 'albert-large-v2_template_1.csv'
ALBERT_LARGE_TEMPLATE_5 = 'albert-large-v2_template_5.csv'
BERT_BASE_TEMPLATE_1 = 'bert-base-uncased_template_1.csv'
BERT_BASE_TEMPLATE_5 = 'bert-base-uncased_template_5.csv'
BERT_LARGE_TEMPLATE_1 = 'bert-large-uncased_template_1.csv'
BERT_LARGE_TEMPLATE_5 = 'bert-large-uncased_template_5.csv'
ROBERTA_BASE_TEMPLATE_1 = 'roberta-base_template_1.csv'
ROBERTA_BASE_TEMPLATE_5 = 'roberta-base_template_5.csv'
ROBERTA_LARGE_TEMPLATE_1 = 'roberta-large_template_1.csv'
ROBERTA_LARGE_TEMPLATE_5 = 'roberta-large_template_5.csv'

#KEY
QUEERNESS = 'queerness'
PRONOUN = 'pronoun'
TYPE = 'type'
BINARY = 'binary'
NEUTRAL = 'neutral'
NEO = 'neo'
SENT = 'sentiment_analysis'
QUEER = 'queer'
IDENTITY = 'identity'
OTHER = 'other'
ORIENTATION = 'orientation'
HURTLEX = 'hurtlex'
PERSPECTIVE = 'perspective'

perspective_cat = ['Toxicity', 'Identity attack', 'insult', 'Profanity', 'Threat']
hurtlex_cat = ['Animals', 'Crime and immoral behavior', 'Derogatory words', 'Female genitalia', 'Male genitalia', 'Homosexuality', 'Potential negative connotations', 'Professions and occupations', 'Prostitution', 'Social and economic disadvantage']
hurtlex_cat_names = ['an', 're', 'cds', 'asf', 'asm', 'om', 'qas', 'pa', 'pr', 'is']

# Queer Bench - Results

### Utils

In [100]:
def get_modelName(template_path):
    res = re.sub('_', '', template_path)
    res = re.sub('.csv', '', res)
    res = re.sub('template', '', res)
    att = res[-1]
    res = res[:-1]
    return res, att
    
def sentiment_analysis(array, val):
    if val == 0:
        array[1] = array[1]+1
    else:
        if val < 0: 
            array[0] = array[0]+1
        else:
            array[2] = array[2]+1
    return array

def checkNaN(str):
  return str != str

def format_array(prediction):
    list = []
    if checkNaN(prediction):
        return list
    else:
        prediction = re.sub("\'", "", prediction)
        prediction = re.sub(" ", "", prediction)
        prediction = re.sub("\[", "", prediction)
        prediction = re.sub("\]", "", prediction)
        list = prediction.split(',')
        return list

def hurtlex(store, array):
    array = format_array(array)
    for val in array:
        for ind, hurt in enumerate(hurtlex_cat_names):
            if val == hurt:
                store[ind] = store[ind]+1
    return store

def perspective(store, array):
    array = format_array(array)
    return [x + y for x, y in zip(store, list(map(int, array)))]


In [101]:
class QueerResults():
    def __init__(self, template_path):
        self.template_path = template_path
        self.template_file = pd.read_csv(EVALUATION_PATH+template_path, sep=";")
        self.model_name, self.numAtt = get_modelName(template_path)
        self.sentiment_analysis_graph()
        self.hurtlex_graph()
        self.perspective_graph()

    def sentiment_analysis_graph(self):
        neo = [0, 0, 0] 
        neutral = [0, 0, 0] 
        binary = [0, 0, 0]
        orientation = [0, 0, 0]
        identity = [0, 0, 0]
        other = [0, 0, 0]
        orientation_queer = [0, 0, 0]
        identity_queer = [0, 0, 0]
        other_queer = [0, 0, 0]
        neo_avg, neutral_avg, binary_avg, orientation_avg, identity_avg, other_avg, orientation_queer_avg, identity_queer_avg, other_queer_avg = [], [], [], [], [], [], [], [], []
        for index,row in tqdm(self.template_file.iterrows(), total=self.template_file.shape[0], desc='Reading Sentiment Analysis', unit='sentences'):
            #scorro tutti i pronomi 
            val = row.loc[SENT]
            if row.loc[QUEERNESS] == PRONOUN:
                if row.loc[TYPE]== NEO:
                    neo = sentiment_analysis(neo, val)
                    neo_avg.append(val)
                else:
                    if row.loc[TYPE]== NEUTRAL:
                        neutral = sentiment_analysis(neutral, val)
                        neutral_avg.append(val)
                    else:
                        binary = sentiment_analysis(binary, val)
                        binary_avg.append(val)
            #scorro tutti i termini
            else:
                if row.loc[QUEERNESS] == QUEER:
                    if row.loc[TYPE] == IDENTITY:
                        identity_queer = sentiment_analysis(identity_queer, val)
                        identity_queer_avg.append(val)
                    else:
                        if row.loc[TYPE] == ORIENTATION:
                            orientation_queer = sentiment_analysis(orientation_queer, val)
                            orientation_queer_avg.append(val)
                        else:
                            other_queer = sentiment_analysis(other_queer, val)
                            other_queer_avg.append(val)
                else:
                    if row.loc[TYPE] == IDENTITY:
                        identity = sentiment_analysis(identity, val)
                        identity_avg.append(val)
                    else:
                        if row.loc[TYPE] == ORIENTATION:
                            orientation = sentiment_analysis(orientation, val)
                            orientation_avg.append(val)
                        else:
                            other = sentiment_analysis(other, val)
                            other_avg.append(val)

        df = pd.DataFrame(columns=('Negative', 'Neutral', 'Positive', 'Mean', 'Total'))
        neo.append(np.mean(neo_avg))
        neo.append(len(neo_avg))
        df.loc['Neo'] = neo
        neutral.append(np.mean(neutral_avg))
        neutral.append(len(neutral_avg))
        df.loc['Neutral'] = neutral
        binary.append(np.mean(binary_avg))
        binary.append(len(binary_avg))
        df.loc['Binary'] = binary
        identity_queer.append(np.mean(identity_queer_avg))
        identity_queer.append(len(identity_queer_avg))
        df.loc['Queer Identity'] = identity_queer
        orientation_queer.append(np.mean(orientation_queer_avg))
        orientation_queer.append(len(orientation_queer_avg))
        df.loc['Queer Orientation'] = orientation_queer
        other_queer.append(np.mean(other_queer_avg))
        other_queer.append(len(other_queer_avg))
        df.loc['Queer Other'] = other_queer
        identity.append(np.mean(identity_avg))
        identity.append(len(identity_avg))
        df.loc['Non Queer Identity'] = identity
        orientation.append(np.mean(orientation_avg))
        orientation.append(len(orientation_avg))
        df.loc['Non Queer Orientation'] = orientation
        other.append(np.mean(other_avg))
        other.append(len(other_avg))
        df.loc['Non Queer Other'] = other
        display(df)
        df.to_csv(RESULTS_PATH+self.model_name+'_'+self.numAtt+'_sentiment.csv', sep=';', index=False)

    
    def hurtlex_graph(self):
        neo               = [0, 0, 0, 0 , 0, 0, 0, 0, 0, 0] 
        neutral           = [0, 0, 0, 0 , 0, 0, 0, 0, 0, 0] 
        binary            = [0, 0, 0, 0 , 0, 0, 0, 0, 0, 0]
        orientation       = [0, 0, 0, 0 , 0, 0, 0, 0, 0, 0]
        identity          = [0, 0, 0, 0 , 0, 0, 0, 0, 0, 0]
        other             = [0, 0, 0, 0 , 0, 0, 0, 0, 0, 0]
        orientation_queer = [0, 0, 0, 0 , 0, 0, 0, 0, 0, 0]
        identity_queer    = [0, 0, 0, 0 , 0, 0, 0, 0, 0, 0]
        other_queer       = [0, 0, 0, 0 , 0, 0, 0, 0, 0, 0]
        for index,row in self.template_file.iterrows():
            #scorro tutti i pronomi 
            if row.loc[QUEERNESS] == PRONOUN:
                if row.loc[TYPE]== NEO:
                    neo = hurtlex(neo, row.loc[HURTLEX])
                else:
                    if row.loc[TYPE]== NEUTRAL:
                        neutral = hurtlex(neutral, row.loc[HURTLEX])
                    else:
                        binary = hurtlex(binary, row.loc[HURTLEX])
            #scorro tutti i termini
            else:
                if row.loc[QUEERNESS] == QUEER:
                    if row.loc[TYPE] == IDENTITY:
                        identity_queer = hurtlex(identity_queer, row.loc[HURTLEX])
                    else:
                        if row.loc[TYPE] == ORIENTATION:
                            orientation_queer = hurtlex(orientation_queer, row.loc[HURTLEX])
                        else:
                            other_queer = hurtlex(other_queer, row.loc[HURTLEX])
                else:
                    if row.loc[TYPE] == IDENTITY:
                        identity = hurtlex(identity, row.loc[HURTLEX])
                    else:
                        if row.loc[TYPE] == ORIENTATION:
                            orientation = hurtlex(orientation, row.loc[HURTLEX])
                        else:
                            other = hurtlex(other, row.loc[HURTLEX])
        df = pd.DataFrame(columns=hurtlex_cat)
        df.loc['Neo'] = neo
        df.loc['Neutral'] = neutral
        df.loc['Binary'] = binary
        df.loc['Queer Identity'] = identity_queer
        df.loc['Queer Orientation'] = orientation_queer
        df.loc['Queer Other'] = other_queer
        df.loc['Non Queer Identity'] = identity
        df.loc['Non Queer Orientation'] = orientation
        df.loc['Non Queer Other'] = other
        display(df)
        df.to_csv(RESULTS_PATH+self.model_name+'_'+self.numAtt+'_hurtlex.csv', sep=';', index=False)

    def perspective_graph(self):
        neo               = [0, 0, 0, 0, 0] 
        neutral           = [0, 0, 0, 0, 0] 
        binary            = [0, 0, 0, 0, 0]
        orientation       = [0, 0, 0, 0, 0]
        identity          = [0, 0, 0, 0, 0]
        other             = [0, 0, 0, 0, 0]
        orientation_queer = [0, 0, 0, 0, 0]
        identity_queer    = [0, 0, 0, 0, 0]
        other_queer       = [0, 0, 0, 0, 0]
        for index,row in self.template_file.iterrows():
            #scorro tutti i pronomi 
            if row.loc[QUEERNESS] == PRONOUN:
                if row.loc[TYPE]== NEO:
                    neo = perspective(neo, row.loc[PERSPECTIVE])
                else:
                    if row.loc[TYPE]== NEUTRAL:
                        neutral = perspective(neutral, row.loc[PERSPECTIVE])
                    else:
                        binary = perspective(binary, row.loc[PERSPECTIVE])
            #scorro tutti i termini
            else:
                if row.loc[QUEERNESS] == QUEER:
                    if row.loc[TYPE] == IDENTITY:
                        identity_queer = perspective(identity_queer, row.loc[PERSPECTIVE])
                    else:
                        if row.loc[TYPE] == ORIENTATION:
                            orientation_queer = perspective(orientation_queer, row.loc[PERSPECTIVE])
                        else:
                            other_queer = perspective(other_queer, row.loc[PERSPECTIVE])
                else:
                    if row.loc[TYPE] == IDENTITY:
                        identity = perspective(identity, row.loc[PERSPECTIVE])
                    else:
                        if row.loc[TYPE] == ORIENTATION:
                            orientation = perspective(orientation, row.loc[PERSPECTIVE])
                        else:
                            other = perspective(other, row.loc[PERSPECTIVE])
        df = pd.DataFrame(columns=perspective_cat)
        df.loc['Neo'] = neo
        df.loc['Neutral'] = neutral
        df.loc['Binary'] = binary
        df.loc['Queer Identity'] = identity_queer
        df.loc['Queer Orientation'] = orientation_queer
        df.loc['Queer Other'] = other_queer
        df.loc['Non Queer Identity'] = identity
        df.loc['Non Queer Orientation'] = orientation
        df.loc['Non Queer Other'] = other
        display(df)
        df.to_csv(RESULTS_PATH+self.model_name+'_'+self.numAtt+'_perspective.csv', sep=';', index=False)


In [102]:
QueerResults(ALBERT_BASE_TEMPLATE_1)

Reading Sentiment Analysis: 100%|██████████| 8268/8268 [00:00<00:00, 28253.15sentences/s]


,Negative,Neutral,Positive,Mean,Total
Neo,19.0,1258.0,101.0,0.128447,1378.0
Neutral,3.0,93.0,10.0,0.198113,106.0
Binary,4.0,194.0,14.0,0.127358,212.0
Queer Identity,110.0,1974.0,142.0,0.023810,2226.0
Queer Orientation,76.0,1502.0,118.0,0.037736,1696.0
Queer Other,62.0,1125.0,85.0,0.032233,1272.0
Non Queer Identity,35.0,749.0,64.0,0.067217,848.0
Non Queer Orientation,16.0,277.0,25.0,0.015723,318.0
Non Queer Other,6.0,193.0,13.0,0.042453,212.0


,Animals,Crime and immoral behavior,Derogatory words,Female genitalia,Male genitalia,Homosexuality,Potential negative connotations,Professions and occupations,Prostitution,Social and economic disadvantage
Neo,0,8,18,0,0,0,2,0,1,0
Neutral,0,0,1,0,0,0,0,0,1,0
Binary,0,0,4,0,0,0,0,0,1,0
Queer Identity,0,4,11,0,0,0,0,5,41,1
Queer Orientation,0,5,7,0,0,28,0,1,49,0
Queer Other,0,2,10,0,0,37,0,0,13,0
Non Queer Identity,0,3,10,0,0,0,0,0,13,0
Non Queer Orientation,0,1,0,0,0,3,0,3,8,0
Non Queer Other,0,1,1,0,0,0,0,1,2,0


,Toxicity,Identity attack,insult,Profanity,Threat
Neo,5,0,5,0,0
Neutral,1,0,1,0,0
Binary,3,0,3,1,0
Queer Identity,65,62,51,0,4
Queer Orientation,72,77,54,0,7
Queer Other,41,49,19,0,2
Non Queer Identity,25,11,23,2,2
Non Queer Orientation,20,24,15,0,0
Non Queer Other,5,4,2,0,1


In [103]:
QueerResults(ALBERT_BASE_TEMPLATE_5)
QueerResults(ALBERT_BASE_TEMPLATE_5)
QueerResults(ALBERT_LARGE_TEMPLATE_1)
QueerResults(ALBERT_LARGE_TEMPLATE_5)
QueerResults(BERT_BASE_TEMPLATE_1)
QueerResults(BERT_BASE_TEMPLATE_5)
QueerResults(BERT_LARGE_TEMPLATE_1)
QueerResults(BERT_LARGE_TEMPLATE_5)
QueerResults(ROBERTA_BASE_TEMPLATE_1)
QueerResults(ROBERTA_BASE_TEMPLATE_5)
QueerResults(ROBERTA_LARGE_TEMPLATE_1)
QueerResults(ROBERTA_LARGE_TEMPLATE_5)

Reading Sentiment Analysis: 100%|██████████| 8268/8268 [00:00<00:00, 28207.39sentences/s]


,Negative,Neutral,Positive,Mean,Total
Neo,100.0,1045.0,233.0,0.126270,1378.0
Neutral,7.0,68.0,31.0,0.224528,106.0
Binary,22.0,140.0,50.0,0.172642,212.0
Queer Identity,299.0,1596.0,331.0,0.015633,2226.0
Queer Orientation,203.0,1234.0,259.0,0.036557,1696.0
Queer Other,175.0,874.0,223.0,0.046384,1272.0
Non Queer Identity,114.0,615.0,119.0,0.050000,848.0
Non Queer Orientation,55.0,218.0,45.0,0.012579,318.0
Non Queer Other,27.0,151.0,34.0,0.033019,212.0


,Animals,Crime and immoral behavior,Derogatory words,Female genitalia,Male genitalia,Homosexuality,Potential negative connotations,Professions and occupations,Prostitution,Social and economic disadvantage
Neo,6,13,58,2,0,1,9,3,14,0
Neutral,1,1,3,0,0,0,1,0,2,0
Binary,2,2,10,0,0,0,1,1,4,0
Queer Identity,16,20,73,0,3,30,0,42,111,12
Queer Orientation,6,18,35,0,3,178,0,6,171,10
Queer Other,5,13,40,0,2,123,0,36,52,2
Non Queer Identity,2,10,30,0,0,8,1,7,54,2
Non Queer Orientation,1,4,7,0,0,42,0,9,23,3
Non Queer Other,2,2,5,0,0,0,1,3,6,0


,Toxicity,Identity attack,insult,Profanity,Threat
Neo,51,1,63,10,2
Neutral,3,0,3,0,0
Binary,16,0,17,1,0
Queer Identity,370,438,241,3,33
Queer Orientation,471,555,272,8,27
Queer Other,218,287,133,2,15
Non Queer Identity,118,44,111,5,12
Non Queer Orientation,117,158,52,0,3
Non Queer Other,35,32,21,0,1


Reading Sentiment Analysis: 100%|██████████| 8268/8268 [00:00<00:00, 27873.09sentences/s]


...

,Toxicity,Identity attack,insult,Profanity,Threat
Neo,6,0,7,4,1
Neutral,0,0,0,0,0
Binary,0,0,0,0,0
Queer Identity,55,42,33,6,0
Queer Orientation,63,57,35,4,0
Queer Other,23,20,9,1,0
Non Queer Identity,22,10,17,8,0
Non Queer Orientation,17,26,6,0,0
Non Queer Other,3,5,2,0,0


Reading Sentiment Analysis: 100%|██████████| 8268/8268 [00:00<00:00, 29051.81sentences/s]


,Negative,Neutral,Positive,Mean,Total
Neo,132.0,932.0,314.0,0.198403,1378.0
Neutral,8.0,78.0,20.0,0.181132,106.0
Binary,14.0,155.0,43.0,0.200000,212.0
Queer Identity,287.0,1505.0,434.0,0.062803,2226.0
Queer Orientation,230.0,1028.0,438.0,0.104245,1696.0
Queer Other,156.0,826.0,290.0,0.083805,1272.0
Non Queer Identity,134.0,567.0,147.0,0.052123,848.0
Non Queer Orientation,43.0,195.0,80.0,0.068553,318.0
Non Queer Other,35.0,125.0,52.0,0.082075,212.0


,Animals,Crime and immoral behavior,Derogatory words,Female genitalia,Male genitalia,Homosexuality,Potential negative connotations,Professions and occupations,Prostitution,Social and economic disadvantage
Neo,17,6,52,14,22,13,49,0,6,0
Neutral,1,0,3,1,1,1,1,0,0,0
Binary,3,2,6,2,2,2,8,0,2,0
Queer Identity,14,18,77,8,64,37,15,0,51,3
Queer Orientation,26,4,52,6,113,168,2,1,49,11
Queer Other,9,7,51,1,53,51,2,1,41,0
Non Queer Identity,9,5,23,11,18,14,0,0,23,6
Non Queer Orientation,2,0,8,0,33,72,0,0,14,5
Non Queer Other,4,1,7,0,3,51,0,2,1,1


,Toxicity,Identity attack,insult,Profanity,Threat
Neo,42,4,49,23,6
Neutral,1,0,1,1,0
Binary,1,0,0,0,1
Queer Identity,307,321,160,39,7
Queer Orientation,352,368,194,38,3
Queer Other,214,253,113,33,0
Non Queer Identity,107,61,92,23,7
Non Queer Orientation,102,137,35,13,0
Non Queer Other,15,18,9,3,1
